In [ ]:
%cd /content
!git clone https://github.com/WongKinYiu/yolov7.git

%cd /content/yolov7/
!pip install -r requirements.txt

/content
Cloning into 'yolov7'...
remote: Enumerating objects: 998, done.
remote: Total 998 (delta 0), reused 0 (delta 0), pack-reused 998
Receiving objects: 100% (998/998), 69.77 MiB | 31.32 MiB/s, done.
Resolving deltas: 100% (465/465), done.
/content/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 8.1 MB/s 


In [ ]:
%cd /content/datasets/
!curl -L "https://app.roboflow.com/ds/Zv0zJmdmoM?key=khaXm3rH5z" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

[Errno 2] No such file or directory: '/content/datasets/'
/content/yolov7
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   893  100   893    0     0    427      0  0:00:02  0:00:02 --:--:--   427
100 43.3M  100 43.3M    0     0  17.9M      0  0:00:02  0:00:02 --:--:--  261M
Archive:  roboflow.zip
replace README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# 다시 

In [ ]:
import argparse
import time
from pathlib import Path
import cv2
import torch
import numpy as np
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel

def letterbox(img, new_shape = (640,640), color = (114,114,114), auto = True, scaleFill = False, scaleup = True, stride = 32):
  shape = img.shape[:2]
  if isinstance(new_shape, int):
    new_shape = (new_shape, new_shape)

  r =min(new_shape[0] / shape[0], new_shape[1] / shape[1])
  if not scaleup:
    r = min(r, 1.0)

  ratio = r, r
  new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
  dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]
  if auto:
    dw, dh = np.mod(dw, stride), np.mod(dh, stride)
  elif scaleFill:
    dw, dh = 0.0, 0.0
    new_unpad = (new_shape[1], new_shape[0])
    ratio = new_shape[1] / shape[1], new_shape[0 / shape[0]]

  dw /= 2
  dh /= 2

  if shape[::-1] != new_unpad:
    img = cv2.resize(img, new_unpad, interpolation = cv2.INTER_LINEAR)
  top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
  left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
  img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value = color)
  return img, ratio, (dw,dh)

In [ ]:
import argparse
import time
from pathlib import Path
import cv2
import torch
import numpy as np
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel

In [ ]:
!curl -L "https://app.roboflow.com/ds/OMbYYMJ864?key=imG6LTt5D4" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip #632증강데이터

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   893  100   893    0     0     27      0  0:00:33  0:00:32  0:00:01   215
100 43.3M  100 43.3M    0     0  1221k      0  0:00:36  0:00:36 --:--:-- 15.2M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/------2022-11-09-165917_png.rf.b03093d6e92f14cb54e3c76cea86970a.jpg  
 extracting: test/images/------2022-11-09-193334_png.rf.f74826de258acd0c8ec731fc5853cef6.jpg  
 extracting: test/images/------2022-11-09-193905_png.rf.1f73c2053f195e7760fa6796afb7405e.jpg  
 extracting: test/images/------2022-11-09-205709_png.rf.75b1ac4c6f245552d30e51271ae3521b.jpg  
 extracting: test/images/------2022-11-09-205910_png.rf.fe6200b66a1702682001067582397a36.jpg  
 extracting: test/images/--

In [ ]:
%cd /content
!git clone https://github.com/WongKinYiu/yolov7

/content
Cloning into 'yolov7'...
remote: Enumerating objects: 998, done.
remote: Total 998 (delta 0), reused 0 (delta 0), pack-reused 998
Receiving objects: 100% (998/998), 69.77 MiB | 17.03 MiB/s, done.
Resolving deltas: 100% (467/467), done.


In [ ]:
%cd /content/yolov7/
!pip install -r requirements.txt

/content/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 39.1 MB/s 


In [ ]:
%cat /content/datasets/data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['baby_fish', 'live_goldfish']

roboflow:
  workspace: project-8hufk
  project: goldfish_oranda_merge
  version: 9
  license: Public Domain
  url: https://universe.roboflow.com/project-8hufk/goldfish_oranda_merge/dataset/9

In [ ]:
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 33.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [ ]:
import yaml

with open('/content/datasets/data.yaml', 'r') as f:
  data = yaml.load(f, Loader=yaml.FullLoader)

print(data)

data['train'] = '/content/datasets/train/images'
data['val'] = '/content/datasets/valid/images'

with open('/content/datasets/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 2, 'names': ['baby_fish', 'live_goldfish'], 'roboflow': {'workspace': 'project-8hufk', 'project': 'goldfish_oranda_merge', 'version': 9, 'license': 'Public Domain', 'url': 'https://universe.roboflow.com/project-8hufk/goldfish_oranda_merge/dataset/9'}}
{'train': '/content/datasets/train/images', 'val': '/content/datasets/valid/images', 'test': '../test/images', 'nc': 2, 'names': ['baby_fish', 'live_goldfish'], 'roboflow': {'workspace': 'project-8hufk', 'project': 'goldfish_oranda_merge', 'version': 9, 'license': 'Public Domain', 'url': 'https://universe.roboflow.com/project-8hufk/goldfish_oranda_merge/dataset/9'}}


In [ ]:
classes_to_filter = ['baby_fish', 'live_goldfish']

opt = {
    'weights' : "/content/drive/MyDrive/product_share/yolov7_632_augmentation_data_epoch_50.pt",
    'yaml' : '/content/datasets/data.yaml',
    'img-size' : 640,
    'conf-thres' : 0.2,
    'iou-thres' : 0.45,
    'device' : '0',
    'classes' : classes_to_filter 
}

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import PIL
import io
import html

#function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  '''
  params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  ''' 
  #decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  #convert byte to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype = np.uint8)
  #decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

def bbox_to_bytes(bbox_aaray):

  #convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_aaray, 'RGBA')
  iobuf = io.BytesIO()
  #format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  #format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()),'utf-8')))

  return bbox_bytes

#JavaScript to properly create out live video stream using out webcam as imput
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom(){
      stream.getVideaoTracks()[0].stop();
      video.remove();
      video = null;
      div = null;
      imgElement = null;
      captyreCanvas = null;
      labelElement = null;
    }

    function onAnimationFrame() {
      if (! shutdown){
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve){
        var result = "";
        if (!shutdown){
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg',0.8)  //여기는 왜 ; 가 안붙었나
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom(){
      if (div !== null){
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'black';
      video.setAttribute('playsinline', '');
      video.onclick = () => {shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
        {video: {facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => {shutdown = true;};
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
          div.appendChild(instruction);
          instruction.onclick = () => {shutdown = true;};

          video.srcObject = stream;
          await video.play();


          captyreCanvas = document.createElement('canvas');
          captyreCanvas.width = 640;  //video.videoWidth;
          captyreCanvas.height = 480;
          window.requestAnimationFrame(onAnimationFrame);
          return stream;
    }
    async function stream_frame(label, imgData){
      if (shutdown){
        removeDom();
        shutdown = false;
        return '';
      }
      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != ""){
        labelElement.innerHTML = label;
      }

      if (imgData != ""){
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
      pendingResolve = resolve;
    });
    shutdown = false;

    return {'create' : preShow - preCreate,
            'show' : preCapture - preShow,
            'capture': Date.now() - preCapture,
            'img' : result};

    }
    ''')
  display(js)

In [ ]:
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
video_stream()
label_html = 'Capturing...'

bbox = ''
count = 0

with torch.no_grad():
  weights, imgsz = opt['weights'], (480,640)
  set_logging()
  device = select_device(opt['device'])
  half = device.type != 'cpu'
  model = attempt_load(weights, map_location=device)
  stride = int(model.stride.max())

  if half:
    model.half()

  names = model.module.names if hasattr(model, 'module') else model.names
  colors = [[random.randint(0,255) for _ in range(3)] for _ in names]
  if device.type != 'cpu':
    model(torch.zeros(1,3,imgsz[0], imgsz[1]).to(device).type_as(next(model.parameters())))
  classes = None
  if opt['classes']:
    classes = []
    for class_name in opt['classes']:
      classes.append(names.index(class_name))

  if classes:
    classes = [i for i in range(len(names)) if i not in classes]

  while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
      break
    img0 = js_to_image(js_reply['img'])
    bbox_array = np.zeros([480,640,4], dtype = np.uint8)
    img = letterbox(img0, imgsz, stride=stride)[0]
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()
    img /= 255.0
    if img.ndimension() ==3:
      img = img.unsqueeze(0)

    t1 = time_synchronized()
    pred = model(img, augment= False)[0]

    pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes = classes, agnostic = False)
    t2 = time_synchronized()
    for i, det in enumerate(pred):
      s = ''
      s += '%gx%g ' % img.shape[2:]
      gn = torch.tensor(img0.shape) [[1, 0, 1, 0]]
      if len(det):
        det[:,:4] = scale_coords(img.shape[2:], det[:,:4], img0.shape).round()

        for c in det[:,-1].unique():
          n = (det[:,-1] == c).sum()
          s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "

        for *xyxy, conf, cls in reversed(det):
          label = f'{names[int(cls)]} {conf:.2f}'
          plot_one_box(xyxy, bbox_array, label=label, color=colors[int(cls)], line_thickness=2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    bbox_bytes = bbox_to_bytes(bbox_array)

    bbox = bbox_bytes

<IPython.core.display.Javascript object>

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
